In [ ]:
# Import necessary Pipeline modules
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.filters.morningstar import Q500US
from quantopian.pipeline.factors import AverageDollarVolume, Latest,Returns
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data import morningstar
from quantopian.interactive.data.eventvestor import dividends_free  as dataset

from zipline.utils.tradingcalendar import trading_day  
import pandas as pd 
import numpy as np
from odo import odo
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [ ]:
# Leamanショック頃
start = "2008-9-29"
end = "2009-3-10"
spy = get_pricing("spy", start_date=start, end_date=end, fields='price', frequency='daily')
research_start_date = spy.pct_change().sort_values().index[0].date()
print "SPYが一番ネガティブになった日： ", research_start_date
research_end_date = pd.date_range(research_start_date, periods=30, freq=trading_day)[-1].date()
print "調査期間：%s ~ %s" % (research_start_date, research_end_date)


In [ ]:
class DividendYield(CustomFactor):  
    """  
    Computes (or rather fetches) dividend yield
    """  
    inputs = [morningstar.valuation_ratios.dividend_yield]
    window_length = 1
    
    def compute(self, today, assets, out, dividend_yield):  
        out[:] = dividend_yield 
        
pipe = Pipeline()
pipe.add(Returns(window_length=2), "Returns")
pipe.add(USEquityPricing.close.latest, "Latest")
pipe.add(MarketCap(), "MarketCap")
pipe.add(Sector(), "Sector")
pipe.add(morningstar.valuation_ratios.pe_ratio.latest , "PER")
pipe.add(morningstar.valuation_ratios.book_value_yield.latest, "PBR") # 本当にPBRかどうか怪しいので確認
pipe.add(DividendYield(), 'DividendYield')

pipe.set_screen(Q500US())        

In [ ]:
result = run_pipeline(pipe, research_start_date, research_end_date)

In [ ]:
returns = result.Returns
sum_returns_by_asset = returns.reset_index().groupby(['level_1']).sum().sort_values(by='Returns', ascending=False)


In [ ]:
sum_returns_by_asset.plot(kind='bar', legend=False)
print "調査期間でのSP500っぽい銘柄のReturn"


In [ ]:
print "調査開始日から90営業日でReturnのCumsumがポジティブになったSP500銘柄数", 
len(sum_returns_by_asset[sum_returns_by_asset.Returns > 0])

In [ ]:
returns_by_sector = result[['Returns', 'Sector']]

sector_mappings = {  
   101.0: "Basic Materials",  
   102.0: "Consumer Cyclical",  
   103.0: "Financial Services",  
   104.0: "Real Estate",  
   205.0: "Consumer Defensive",  
   206.0: "Healthcare",  
   207.0: "Utilites",  
   308.0: "Communication Services",  
   309.0: "Energy",  
   310.0: "Industrials",  
   311.0: "Technology"  
}  
sum_returns_by_sector = returns_by_sector.reset_index().groupby(['level_1']).mean().groupby(['Sector']).mean().rename(index= sector_mappings)
sum_returns_by_sector.sort_values(by='Returns', ascending=False).plot(kind='bar')
plt.title("return by setcor")


In [ ]:
returns_by_sector[returns_by_sector.Sector==207].reset_index().groupby("level_1").sum().sort_values(by="Returns")['Returns'].plot(kind='bar')
plt.title("Utilites setcors")

# for sid in returns_by_sector[returns_by_sector.Sector==207].reset_index().groupby("level_1").sum().index:
#     print sid, sid.security_name

In [ ]:
returns_by_pbr = result[['Returns', 'PBR', 'Sector']]
returns_by_pbr = returns_by_pbr.reset_index().groupby(['level_1']).mean()
returns_by_pbr.plot(x='PBR', y='Returns', kind='scatter', c='Sector', cmap=cm.Reds)
plt.title("PBR vs Returns")

In [ ]:
returns_by_per = result[['Returns', 'PER', 'DividendYield']]
returns_by_per = returns_by_per.reset_index().groupby(['level_1']).mean()#.groupby(['Sector']).sum().rename(index= sector_mappings)
ax = returns_by_per.plot(x='PER', y='Returns', kind='scatter')#, c='DividendYield', cmap=cm.Reds)

plt.xlim(0, 30)
plt.title("PER vs Return")



In [ ]:

return_by_dy = result[['Returns', 'DividendYield', 'PER']]
return_by_dy = return_by_dy.reset_index().groupby(['level_1']).mean()

return_by_dy.plot(x='DividendYield', y='Returns', kind='scatter',)
                 #c = 'PER', cmap=cm.Reds)
plt.title("Dividend Yield vs Returns")




In [ ]:
return_3 = result[['Returns', 'DividendYield', 'PER', "PBR", "MarketCap", ]]
return_3 = return_3.reset_index().groupby(['level_1']).mean()

fig = plt.figure()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)


ax = fig.add_subplot(1,4,1)
ax.scatter(return_3.DividendYield, return_3.Returns)
ax.set_title('Dividend')

ax = fig.add_subplot(1,4,2)
ax.scatter(return_3.PER, return_3.Returns)
ax.set_title('PER')

ax = fig.add_subplot(1,4,3)
ax.scatter(return_3.PBR, return_3.Returns)
ax.set_title('PBR')

ax = fig.add_subplot(1,4,4)
ax.scatter(return_3.PBR, return_3.Returns)
ax.set_title('MarketCap')



In [ ]:
df = return_3.sort_values(by = 'Returns', ascending=False)

idx = pd.IndexSlice
result.loc[idx[:, df[df.Returns > 0].index], "Returns"]

df[df.Returns > 0]#.reset_index()


